In [1]:
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set();

from dpipe.io import load

In [2]:
def get_stats_spottune(exp_path, fold='inference'):
    p = torch.load(Path(exp_path) / f'policy_{fold}_record/policy_record')
    f = open(Path(exp_path) / f'policy_{fold}_record/iter_record', 'r')
    n_iter = f.read()
    f.close()
    record = (p / int(n_iter)).detach().numpy()
    return record

In [3]:
ns = np.arange(25)

# '8' has some problems with NaN policy...
slices2folder = {'5': '1_48', '12': '1_24', '24': '1_12', '45': '1_6',
                 '90': '1_3', '270': '1', '540': '2', '800': '3'}
path_template = '/gpfs/data/gpfs0/b.shirokikh/experiments/da/miccai2021_spottune/finetune/{}/spottune_new/'

records = defaultdict(list)
for k, v in slices2folder.items():
    for n in ns:
        records[k].append(get_stats_spottune(Path(path_template.format(v)) / f'experiment_{n}'))

In [4]:
records_mean = {k: 1 - np.mean(v, axis=0) for k, v in records.items()}

In [ ]:
# TODO: switch to new (~lighter) palette
# sns.cubehelix_palette(start=2, rot=0, dark=0, light=.95, reverse=True)
# sns.color_palette("dark:salmon_r")  # remove _r
# 

In [5]:
def floats2colors(floats, bins=256):
    palette = sns.color_palette(palette='magma', n_colors=bins).as_hex()
    ints = np.int64(floats * (bins - 1))
    colors = [palette[i] for i in ints]
    return colors

### colors per blocks

In [6]:
colors = {k: floats2colors(v) for k, v in records_mean.items()}
print(-1, *[f'{k:10s}' for k in colors.keys()])
for i in range(1, len(colors['5']) + 1):
    print(f'{i:2d}', *[f'{v[i - 1]:10s}' for _, v in colors.items()])

-1 5          12         24         45         90         270        540        800       
 1 #000004    #010005    #010005    #050416    #000004    #050416    #000004    #671b80   
 2 #fcfdbf    #fecd90    #fde7a9    #fcfdbf    #fcfdbf    #fcfdbf    #fcfdbf    #fcfdbf   
 3 #000004    #010005    #621980    #221150    #06051a    #02020b    #120d31    #f3655c   
 4 #000004    #010005    #241253    #1c1044    #120d31    #19103f    #271258    #fd9467   
 5 #010106    #000004    #050416    #02020d    #000004    #020109    #000004    #feae77   
 6 #fcecae    #febb81    #fcfdbf    #fcfdbf    #fcfdbf    #fcfdbf    #fcfdbf    #fcfdbf   
 7 #000004    #000004    #19103f    #040414    #000004    #020109    #150e38    #df4a68   
 8 #010005    #000004    #a8327d    #902a81    #221150    #6a1c81    #5f187f    #fcf0b2   
 9 #000004    #000004    #160f3b    #150e38    #020109    #110c2f    #0b0924    #fdda9c   
10 #fb8761    #ee5b5e    #fcfbbd    #fcfbbd    #fec488    #fcecae    #feb078    #fcfbbd   

### colorbar

In [ ]:
colors = floats2colors(np.linspace(0, 1, num=11))
for i, c in enumerate(colors, start=1):
    print(i, c)